# ADS 509 Module 1 Part 2: Scraping Lyrics 

This notebook holds part two of the assignment for ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the songs and store them in text files. 

The lyrics for an artist on AZLyrics can be accessed via a link like this: https://www.azlyrics.com/r/robyn.html. You will need to find the links for your two artists.

## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import requests
import os
import time
from bs4 import BeautifulSoup
from collections import defaultdict

In [2]:
# Use this cell for any import statements you add
import re
import shutil
import random

In [3]:
artists = {'robyn':"https://www.azlyrics.com/r/robyn.html",
           'cher':"https://www.azlyrics.com/c/cher.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: <!-- Delete this comment and put your answer here. --> This scraping is allowed. The `robots.txt` file disallows scraping the lyricsdb and song portions of the file hierarchy. Everything else, including the artist and lyric pages, is allowed. 


In [4]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    

Let's make sure we have enough lyrics pages to scrape. 

In [5]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [6]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For robyn we have 103.
The full pull will take for this artist will take 0.29 hours.
For cher we have 318.
The full pull will take for this artist will take 0.88 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [7]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [13]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [14]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :

    # TK stopped here in the cleaning
    
    os.mkdir("lyrics/" + artist)
    pages = 0
    
    for page in lyrics_pages[artist] :
        
        page_name = page.replace("/lyrics/","").replace(".html","")
        print(f"Requesting: {page_name}")
        
        # make actual URL
        url = url_stub + page
        
        # request
        r = requests.get(url)
        time.sleep(5 + 10*random.random()) # Put in a random wait

        pages += 1
        
        if r.status_code != 200 :
            print(f"Got a bad status code ({r.status_code}) on {page}.")
        else :
            soup = BeautifulSoup(r.text,"html.parser")
       
            output_filename = "lyrics/" + artist + "/" + generate_filename_from_url(page)

            # First let's get the song title. 
            for item in soup.find_all("b") : 
                if '"' in item.text :
                    title = item.text
                    break
                    # First one with quotes is song. Second is album.
        
            # Let's get the lyrics
            hit_ringtone = False
            for item in soup.find_all("div") : 
                
                # Lyrics are the div after the ringtone
                if hit_ringtone : 
                    break

                if "class" in item.attrs : 
                    if "ringtone" in item.attrs["class"] : 
                        hit_ringtone = True

            lyrics = item.text
            
            if title == '"It\'s Not Unusual"' :
                print("Oops, looks like we went too fast. Getting Tom Jones back.")
                time.sleep(360 + random.random()*80) # see if a long sleep works
        
            with open(output_filename,'w') as ofile :
                ofile.write(title + "\n\n")
                ofile.write(lyrics)
            
        if pages == 10 :
            break
#            
#            break
                
#        time.sleep(random.random()*40)
    
    # Sleep between artists
    time.sleep(60)
    total_pages += pages
    


Requesting: robyn/bumpyride
Requesting: robyn/inmyheart
Requesting: robyn/youvegotthatsomething
Requesting: robyn/doyouknowwhatittakes
Requesting: robyn/thelasttime
Requesting: robyn/justanothergirlfriend
Requesting: robyn/dontwantyouback
Requesting: robyn/doyoureallywantmeshowrespect
Requesting: robyn/how
Requesting: robyn/herewego
Requesting: cher/allireallywanttodo
Requesting: cher/igotosleep
Requesting: cher/needlesandpins
Requesting: cher/dontthinktwice
Requesting: cher/dreambaby
Requesting: cher/thebellsofrhymney


ConnectionError: HTTPSConnectionPool(host='www.azlyrics.com', port=443): Max retries exceeded with url: /lyrics/cher/thebellsofrhymney.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe2d16d5d90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")